In [1]:
import numpy as np
from matplotlib import pyplot as plt
import time

In [11]:
# n_features = 16000
# n_samples = 15000
n_features = 2
n_samples = 10
n_classes = 2
X = np.random.randn(n_features, n_samples)
y = np.random.randint(0, n_classes, n_samples)
y = np.array([1 if i == 1 else -1 for i in y])
weights = np.random.randn(n_samples)


# Their method

In [12]:
s_t = time.time()

total_pos, total_neg = 0, 0
for w, label in zip(weights, y):
    if label == 1:
        total_pos += w
    else:
        total_neg += w

classifiers = []
total_features = X.shape[0]
for index, feature in enumerate(X):
    if len(classifiers) % 1000 == 0 and len(classifiers) != 0:
        print("Trained %d classifiers out of %d" % (len(classifiers), total_features))

    applied_feature = sorted(zip(weights, feature, y), key=lambda x: x[1])

    pos_seen, neg_seen = 0, 0
    pos_weights, neg_weights = 0, 0
    min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None
    current_idx = 0
    for w, f, label in applied_feature:
        # min(all before current example are positive and all after are negative, all before current example are negative and all after are positive)
        # error = sum of weights of misclassified examples
        error = min(neg_weights + total_pos - pos_weights, pos_weights + total_neg - neg_weights)
        if error < min_error:
            min_error = error
            # best_feature = features[index]
            best_feature = (current_idx, f)
            current_idx += 1
            best_threshold = f
            best_polarity = 1 if pos_seen > neg_seen else -1

        if label == 1:
            pos_seen += 1
            pos_weights += w
        else:
            neg_seen += 1
            neg_weights += w
    
    # clf = WeakClassifier(best_feature[0], best_feature[1], best_threshold, best_polarity)
    clf = best_feature, best_threshold, best_polarity, min_error
    classifiers.append(clf)

print("Time taken: %f seconds" % (time.time() - s_t))

Time taken: 0.000314 seconds


# Mine

In [13]:
s_t = time.time()
classifiers2 = []
total_features2 = X.shape[0]
# TODO parallize this
for index, feature in enumerate(X):
    if len(classifiers2) % 1000 == 0 and len(classifiers2) != 0:
        print("Trained %d classifiers out of %d" % (len(classifiers2), total_features2))
        
    pos_weights, neg_weights = 0, 0
    min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None

    # sort by feature value first, then by weight, then by label
    sorting_indecies = np.lexsort((y, weights, feature))
    
    # s_w: srorted weights, s_f: sorted features, s_y: sorted labels
    s_w, s_f, s_y = weights[sorting_indecies], feature[sorting_indecies], y[sorting_indecies]

    ## left = -1, right = 1
    # negative weights
    neg_weights = np.cumsum(s_w * (s_y == -1))
    # positive weights (reversed)
    pos_weights = np.flip(np.cumsum(np.flip(s_w * (s_y == 1))))
    pos_weights2 = np.concatenate((pos_weights[1:], [0]))
    
    idx1 = np.argmin(neg_weights + pos_weights2)
    
    min_error1 = neg_weights[idx1] + pos_weights2[idx1]
    # best_feature1 = (idx1, s_f[idx1])
    # best_threshold1 = s_f[idx1]
    try:best_threshold1 = s_f[idx1+1], best_feature1 = (idx1, s_f[idx1+1])
    except:best_threshold1 = s_f[idx1], best_feature1 = (idx1, s_f[idx1])
    
    ## left = 1, right = -1
    # repeat for s_y ==1, s_y == 1
    neg_weights_rl = np.flip(np.cumsum(np.flip(s_w * (s_y == -1))))
    neg_weights_rl = np.concatenate((neg_weights_rl[1:], [0]))
    pos_weights_rl = np.cumsum(s_w * (s_y == 1))

    idx2 = np.argmin(neg_weights_rl + pos_weights_rl)
    min_error2 = neg_weights_rl[idx2] + pos_weights_rl[idx2]
    # best_feature2 = (idx2, feature2[idx2])
    # best_threshold2 = feature2[idx2]
    try:best_threshold2 = s_f[idx2+1], best_feature2 = (idx2, s_f[idx2+1])
    except:best_threshold2 = s_f[idx2], best_feature2 = (idx2, s_f[idx2])

    if min_error1 < min_error2:
        min_error = min_error1
        best_feature = best_feature1
        best_threshold = best_threshold1
        best_polarity = -1
    else:
        min_error = min_error2
        best_feature = best_feature2
        best_threshold = best_threshold2
        best_polarity = 1

    clf2 = best_feature, best_threshold, best_polarity, min_error
    classifiers2.append(clf2)

    

print("Time taken: %f seconds" % (time.time() - s_t))

Time taken: 0.000573 seconds


In [14]:
# for el in classifiers:
#     print(el)
# print()
# for el in classifiers2:
#     print(el)

print(classifiers[0])
print(classifiers2[0])

((3, 1.6154135590639638), 1.6154135590639638, 1, -1.711756830617832)
(1.6154135590639638, (8, 1.6154135590639638), -1, -1.711756830617832)
